<a href="https://colab.research.google.com/github/bitanb1999/Explain_Depression/blob/main/models/Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/Kayal-Sampath/detecting-signs-of-depression-from-social-media-postings/main/dev.tsv
!wget https://raw.githubusercontent.com/Kayal-Sampath/detecting-signs-of-depression-from-social-media-postings/main/train.tsv
!wget https://raw.githubusercontent.com/Kayal-Sampath/detecting-signs-of-depression-from-social-media-postings/main/test.tsv

--2022-09-01 17:18:55--  https://raw.githubusercontent.com/Kayal-Sampath/detecting-signs-of-depression-from-social-media-postings/main/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4048219 (3.9M) [text/plain]
Saving to: ‘dev.tsv’

dev.tsv             100%[===================>]   3.86M  --.-KB/s    in 0.07s   

2022-09-01 17:18:55 (55.5 MB/s) - ‘dev.tsv’ saved [4048219/4048219]

--2022-09-01 17:18:55--  https://raw.githubusercontent.com/Kayal-Sampath/detecting-signs-of-depression-from-social-media-postings/main/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP re

In [26]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
dev=pd.read_csv("dev.tsv", sep='\t')
train=pd.read_csv("train.tsv", sep='\t')
test=pd.read_csv("test.tsv", sep='\t')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
label_dict={}
l=0;
for i in ['not depression','moderate','severe']:
  label_dict[i]=l
  l+=1
label_dict

{'not depression': 0, 'moderate': 1, 'severe': 2}

In [28]:
train['Label']=train['Label'].apply(lambda x:label_dict[x])
dev['Label']=dev['Label'].apply(lambda x:label_dict[x])

In [29]:
train=train[['Text_data','Label']]
dev=dev[['Text data','Label']]

In [30]:
dev.rename(columns={'Text data':'Text_data'},inplace=True)
#############################
# concat all
df=pd.concat([train,dev])
df.reset_index(drop=True,inplace=True)

from nltk.stem import PorterStemmer
stop_words = set(stopwords.words("english"))
stemer = PorterStemmer()
def remove_SW_Stem(text):
 text=[stemer.stem(words) for words in text.split(" ") if words not in stop_words]
 return " ".join(text)
special_chars = re.compile("#+_]")
add_space = re.compile("[/(){}\[\]\\@;]")
def clean_text(text):
 text=text.lower()
 text = add_space.sub(" ",text)
 text = special_chars.sub(" ",text)
 text = remove_SW_Stem(text)
 return text

In [31]:
df['Text_data']=df['Text_data'].apply(lambda text:clean_text(text))

In [32]:
train = df[~df['Label'].isna()]
X_train, X_val, y_train, y_val = train_test_split(train, train['Label'], test_size=0.2, random_state=42)

In [33]:
test.rename(columns={'text data':'Text_data'},inplace=True)

In [34]:
from keras.preprocessing import sequence
from keras.preprocessing import text

# Limit on the number of features to K features.
TOP_K = 20000

# Limit on the length of text sequences. 
# Sequences longer than this will be truncated.
# and less than it will be padded
MAX_SEQUENCE_LENGTH = 512

class CustomTokenizer:
    def __init__(self, train_texts):
        self.train_texts = train_texts
        self.tokenizer = Tokenizer(num_words=TOP_K)
        
    def train_tokenize(self):
        # Get max sequence length.
        max_length = len(max(self.train_texts , key=len))
        self.max_length = min(max_length, MAX_SEQUENCE_LENGTH)
    
        # Create vocabulary with training texts.
        self.tokenizer.fit_on_texts(self.train_texts)
        
    def vectorize_input(self, tweets):
        # Vectorize training and validation texts.
        
        tweets = self.tokenizer.texts_to_sequences(tweets)
        # Fix sequence length to max value. Sequences shorter than the length are
        # padded in the beginning and sequences longer are truncated
        # at the beginning.
        tweets = sequence.pad_sequences(tweets, maxlen=self.max_length, truncating='post',padding='post')
        return tweets
    
tokenizer = CustomTokenizer(train_texts = X_train['Text_data'])
# fit o the train
tokenizer.train_tokenize()
tokenized_train = tokenizer.vectorize_input(X_train['Text_data'])
tokenized_val = tokenizer.vectorize_input(X_val['Text_data'])
tokenized_test = tokenizer.vectorize_input(test['Text_data'])

In [17]:
import os
import tqdm
import requests
import zipfile
URL = "http://nlp.stanford.edu/data/glove.42B.300d.zip"

def fetch_data(url=URL, target_file='glove.zip', delete_zip=False):
    #if the dataset already exists exit
    if os.path.isfile(target_file):
        print("datasets already downloded :) ")
        return

    #download (large) zip file
    #for large https request on stream mode to avoid out of memory issues
    #see : http://masnun.com/2016/09/18/python-using-the-requests-module-to-download-large-files-efficiently.html
    print("**************************")
    print("  Downloading zip file")
    print("  >_<  Please wait >_< ")
    print("**************************")
    response = requests.get(url, stream=True)
    #read chunk by chunk
    handle = open(target_file, "wb")
    for chunk in tqdm.tqdm(response.iter_content(chunk_size=512)):
        if chunk:  
            handle.write(chunk)
    handle.close()  
    print("  Download completed ;) :") 
    #extract zip_file
    zf = zipfile.ZipFile(target_file)
    print("1. Extracting {} file".format(target_file))
    zf.extractall()
    if delete_zip:
        print("2. Deleting {} file".format(dataset_name+".zip"))
        os.remove(path=zip_file)

fetch_data()

**************************
  >_<  Please wait >_< 
**************************


3667580it [05:52, 10403.36it/s]


  Download completed ;) :
1. Extracting glove.zip file


In [18]:
glove_file = "glove.42B.300d.txt"
import tqdm

EMBEDDING_VECTOR_LENGTH = 100 # <=200
def construct_embedding_matrix(glove_file, word_index):
    embedding_dict = {}
    with open(glove_file,'r') as f:
        for line in f:
            values=line.split()
            # get the word
            word=values[0]
            if word in word_index.keys():
                # get the vector
                vector = np.asarray(values[1:], 'float32')
                embedding_dict[word] = vector
    ###  oov words (out of vacabulary words) will be mapped to 0 vectors

    num_words=len(word_index)+1
    #initialize it to 0
    embedding_matrix=np.zeros((num_words, EMBEDDING_VECTOR_LENGTH))

    for word,i in tqdm.tqdm(word_index.items()):
        if i < num_words:
            vect=embedding_dict.get(word, [])
            if len(vect)>0:
                embedding_matrix[i] = vect[:EMBEDDING_VECTOR_LENGTH]
    return embedding_matrix
  
embedding_matrix =  construct_embedding_matrix(glove_file, tokenizer.tokenizer.word_index)

100%|██████████| 17922/17922 [00:00<00:00, 404654.92it/s]


In [19]:
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [20]:
from keras.layers import Embedding,LSTM,Dense,Dropout
model=Sequential()

embedding=Embedding(len(tokenizer.tokenizer.word_index)+1, # number of unique tokens
                    EMBEDDING_VECTOR_LENGTH, #number of features
                    embeddings_initializer=Constant(embedding_matrix), # initialize 
                    input_length=MAX_SEQUENCE_LENGTH, 
                    trainable=False)

model.add(embedding)
model.add(Dropout(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.5))
model.add(Dense(1, activation='softmax'))
# compile the model
optimzer = Adam(clipvalue=0.5) # clip value to avoid the gradient exploding

model.compile(optimizer=optimzer, 
              loss='binary_crossentropy', 
              metrics=['acc',f1_m,precision_m, recall_m])

# fit the model
history = model.fit(tokenized_train, y_train, 
                    batch_size=32, 
                    epochs=10, 
                    validation_data=(tokenized_val,y_val), 
                    verbose=2)

Epoch 1/10
335/335 - 247s - loss: 0.5057 - acc: 0.6209 - f1_m: 0.8304 - precision_m: 0.7144 - recall_m: 1.0000 - val_loss: 0.4772 - val_acc: 0.6202 - val_f1_m: 0.8336 - val_precision_m: 0.7179 - val_recall_m: 1.0000 - 247s/epoch - 737ms/step
Epoch 2/10
335/335 - 241s - loss: 0.4893 - acc: 0.6209 - f1_m: 0.8313 - precision_m: 0.7143 - recall_m: 1.0000 - val_loss: 0.4795 - val_acc: 0.6202 - val_f1_m: 0.8336 - val_precision_m: 0.7179 - val_recall_m: 1.0000 - 241s/epoch - 719ms/step
Epoch 3/10
335/335 - 250s - loss: 0.4878 - acc: 0.6209 - f1_m: 0.8310 - precision_m: 0.7145 - recall_m: 1.0000 - val_loss: 0.4775 - val_acc: 0.6202 - val_f1_m: 0.8336 - val_precision_m: 0.7179 - val_recall_m: 1.0000 - 250s/epoch - 746ms/step
Epoch 4/10
335/335 - 240s - loss: 0.4875 - acc: 0.6209 - f1_m: 0.8305 - precision_m: 0.7144 - recall_m: 1.0000 - val_loss: 0.4779 - val_acc: 0.6202 - val_f1_m: 0.8336 - val_precision_m: 0.7179 - val_recall_m: 1.0000 - 240s/epoch - 716ms/step
Epoch 5/10
335/335 - 242s - loss

In [21]:
loss, accuracy, f1_score, precision, recall = model.evaluate(tokenized_val, y_val, verbose=0)

In [22]:
f1_score

0.8336421251296997

In [37]:
preds=model.predict(tokenized_test,verbose=0)

In [38]:
test['Preds']=preds

In [41]:
test['Preds'].unique()

array([1.], dtype=float32)